In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame()
for century in range(1, 11):
    df2 = pd.DataFrame()
    df2['prophecy'] = pd.read_csv(f'./nostradamus_docs/nostradamus_century_{century}.txt', sep = '\t', header = None)[0]
    df2['century'] = century
    df = pd.concat([df, df2], axis = 0, sort = False)    

In [3]:
df.head()

,prophecy,century
0,Century I,1
1,1,1
2,Sitting alone at night in secret study;,1
3,it is placed on the brass tripod.,1
4,A slight flame comes out of the emptiness and,1


In [4]:
df = df[(['Century' not in s for s in df['prophecy']])].reset_index()
df.drop('index', axis = 1, inplace = True)

In [5]:
df = df[np.mod(np.arange(df.index.size), 5) != 0].reset_index()
df.drop('index', axis = 1, inplace = True)

In [6]:
df.loc[df.index % 4 == 0, 'line_num'] = 1
df.loc[df.index % 4 == 1, 'line_num'] = 2
df.loc[df.index % 4 == 2, 'line_num'] = 3
df.loc[df.index % 4 == 3, 'line_num'] = 4

In [7]:
n = 0
century = 1
boolean_list = (df.index % 4 == 0).tolist()
for i, boolean in enumerate(boolean_list):
    if boolean and df.loc[i, 'century'] == century:
        n+=1
        df.loc[i, 'prophecy_num'] = n
    elif boolean:
        century+=1
        n = 1
        df.loc[i, 'prophecy_num'] = n
    else:
        df.loc[i, 'prophecy_num'] = n

In [8]:
df = df.groupby(['century', 'prophecy_num'])['prophecy'].apply(lambda x: '  '.join(x)).reset_index()

In [9]:
df['century'] = df['century'].astype(float)

In [10]:
len(df)

942

In [11]:
df.head()

,century,prophecy_num,prophecy
0,1.0,1.0,Sitting alone at night in secret study; it is...
1,1.0,2.0,The wand in the hand is placed in the middle o...
2,1.0,3.0,When the litters are overturned by the whirlwi...
3,1.0,4.0,In the world there will be made a king who wi...
4,1.0,5.0,They will be driven away for a long drawn out ...


In [12]:
from pymongo import MongoClient

In [13]:
client = MongoClient()
db = client.nostradamus
db.create_collection('prophecy')

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'nostradamus'), 'prophecy')

In [14]:
# db.drop_collection('prophecy')

In [15]:
client.address

('localhost', 27017)

In [16]:
client.list_database_names()

['MyDatabase', 'admin', 'config', 'local', 'nostradamus']

In [17]:
db.list_collection_names()

['prophecy']

In [18]:
col = db.get_collection('prophecy')

In [19]:
col.estimated_document_count()

0

In [20]:
for i, prophecy in enumerate(df['prophecy']):
    doc = {'prophecy': prophecy, 'century': df['century'][i], 'prophecy_num': df['prophecy_num'][i]}
    col.insert_one(doc)

In [21]:
col.estimated_document_count()

942

In [22]:
# for doc in col.find():
#     print(doc)

In [23]:
df = pd.DataFrame(list(col.find()))
df.drop('_id', axis = 1, inplace = True)

In [24]:
df.head()

,century,prophecy,prophecy_num
0,1.0,Sitting alone at night in secret study; it is...,1.0
1,1.0,The wand in the hand is placed in the middle o...,2.0
2,1.0,When the litters are overturned by the whirlwi...,3.0
3,1.0,In the world there will be made a king who wi...,4.0
4,1.0,They will be driven away for a long drawn out ...,5.0


In [25]:
df.to_pickle('prophecy_df.pkl')